In [172]:
# Import and create dfs.
import pandas as pd
import numpy as np
import re

file_assetList = 'Input/IEA equipment inventory_20190925.xlsx'
df_input = pd.read_excel(file_assetList, sheet_name='Sheet1')
#display(df_input.columns)

In [191]:
#display(df_input)

In [183]:
#Extract Floor PDU Circuits

#Check for non-conforming Floor PDU Circuit names (e.g. Not PDU-01-6L1)
print('The following are non-conforming Floor PDU Circuit names, e.g. NOT PDU-1-6L1/23')
display(df_input.loc[~df_input['PDU No. / Socket No.'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No.'])
display(df_input.loc[~df_input['PDU No. / Socket No..1'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..1'])
display(df_input.loc[~df_input['PDU No. / Socket No..2'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..2'])
display(df_input.loc[~df_input['PDU No. / Socket No..3'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'PDU No. / Socket No..3'])
display(df_input.loc[~df_input['Remarks'].str.match('PDU-\d+-\d+[L][123]\s*[-/]\s*\d+', na=True, case=False), 'Remarks'])


df_pdus = df_input['PDU No. / Socket No.'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus[0].sort_values().dropna().unique()

df_pdus1 = df_input['PDU No. / Socket No..1'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus1[0].sort_values().dropna().unique()

df_pdus2 = df_input['PDU No. / Socket No..2'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus2[0].sort_values().dropna().unique()

df_pdus3 = df_input['PDU No. / Socket No..3'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus3[0].sort_values().dropna().unique()

df_pdus4 = df_input['Remarks'].str.extract('(\w+-\d+-\d+[Ll][123])[-/](\d+)')
df_pdus4[0].sort_values().dropna().unique()

list_pdus = np.append(df_pdus[0].sort_values().dropna().unique(), df_pdus1[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus2[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus3[0].sort_values().dropna().unique())
list_pdus = np.append(list_pdus, df_pdus4[0].sort_values().dropna().unique())

#df_racks = pd.DataFrame(df_input['IEA Rack'].sort_values().dropna().unique(), columns=['Name'])

df_pdus_final = pd.DataFrame(list_pdus, columns=['Name'])
df_circuits = pd.DataFrame(df_pdus_final['Name'].sort_values().unique(), columns=['Circuit'])
display(df_circuits)

df_circuits.to_csv("Output/Circuits.csv", ",", index=False)

The following are non-conforming Floor PDU Circuit names, e.g. NOT PDU-1-6L1/23


1                  ATS-PORT-2
2                  ATS-PORT-3
3                  ATS-PORT-5
41     ATS-2 power bar slot 1
42      ATS-2 power bar sot 3
48         Using power supply
94                 ATS-Slot 2
98                 ATS-Slot 2
102                ATS-slot 7
129                ATS-Slot 2
139                ATS-slot 6
145               ATS- slot 2
Name: PDU No. / Socket No., dtype: object

81    PDU-1-3L2 /
82    PDU-1-3L2 /
83    PDU-1-3L2 /
84    PDU-1-3L2 /
86    PDU-1-4L1 /
Name: PDU No. / Socket No..1, dtype: object

3             ATS-PORT-6
48    Using power supply
Name: PDU No. / Socket No..2, dtype: object

Series([], Name: PDU No. / Socket No..3, dtype: object)

81     PDU-2-3L2 /
82     PDU-2-3L2 /
83     PDU-2-3L2 /
84     PDU-2-3L2 /
86     PDU-2-4L1 /
134    TO BE DECOM
Name: Remarks, dtype: object

,Circuit
0,PDU-01-09L3
1,PDU-01-10L2
2,PDU-01-10L3
3,PDU-01-4L2
4,PDU-01-6L1
5,PDU-01-8L2
6,PDU-01-9L1
7,PDU-01-9L3
8,PDU-02-09L3
9,PDU-02-10L2


In [190]:
#Create "Add Rack PDUs" Import Sheet

#Extracting circuit info to create standard circuits
df_circuits['Panel'] = df_circuits['Circuit'].str.extract('PDU-(\d+)-.*',  flags=re.IGNORECASE)
df_circuits['Panel'] = df_circuits['Panel'].fillna(0).astype(int)
df_circuits['Line'] = df_circuits['Circuit'].str.extract('PDU-\d+-(\d+).*',  flags=re.IGNORECASE)
df_circuits['Line'] = df_circuits['Line'].fillna(0).astype(int)
df_circuits['Phase'] = df_circuits['Circuit'].str.extract('PDU-\d+-\d+L(\d+)',  flags=re.IGNORECASE)
df_circuits['Phase'] = df_circuits['Phase'].fillna(0).astype(int)

df_circuits['Circuit Name'] = "PDU-" + df_circuits['Panel'].astype(str) + "-" + df_circuits['Line'].astype(str) + "L" + df_circuits['Phase'].astype(str)
display(df_circuits['Circuit Name'].sort_values().dropna().unique())

df_rpdus = pd.DataFrame(df_circuits['Circuit Name'].sort_values().dropna().unique(), columns=['Name'])
df_rpdus['PDU Name'] = ""
df_rpdus['Make'] = "Raritan"
df_rpdus['Model'] = ""
df_rpdus['Cabinet'] = ""
df_rpdus['Depth Position'] = ""
df_rpdus['Cabinet Side'] = ""
df_rpdus['U Position'] = ""
df_rpdus['IP Address'] = ""
df_rpdus['Proxy ID'] = ""
df_rpdus.to_csv("Output/Circuits_and_RackPDU.csv", ",", index=False)

array(['PDU-1-10L1', 'PDU-1-10L2', 'PDU-1-10L3', 'PDU-1-1L1', 'PDU-1-1L3',
       'PDU-1-2L1', 'PDU-1-2L2', 'PDU-1-2L3', 'PDU-1-3L1', 'PDU-1-3L3',
       'PDU-1-4L2', 'PDU-1-5L1', 'PDU-1-5L3', 'PDU-1-6L1', 'PDU-1-6L2',
       'PDU-1-6L3', 'PDU-1-7L1', 'PDU-1-7L2', 'PDU-1-7L3', 'PDU-1-8L2',
       'PDU-1-9L1', 'PDU-1-9L2', 'PDU-1-9L3', 'PDU-2-10L1', 'PDU-2-10L2',
       'PDU-2-1L1', 'PDU-2-1L3', 'PDU-2-2L1', 'PDU-2-2L2', 'PDU-2-2L3',
       'PDU-2-3L1', 'PDU-2-3L3', 'PDU-2-4L2', 'PDU-2-5L1', 'PDU-2-5L3',
       'PDU-2-6L2', 'PDU-2-6L3', 'PDU-2-7L1', 'PDU-2-7L2', 'PDU-2-7L3',
       'PDU-2-8L2', 'PDU-2-9L1', 'PDU-2-9L2', 'PDU-2-9L3'], dtype=object)

In [116]:
#Extract Rack from df_input
#Create Add Cabinet import sheet
print('List of racks', df_input['IEA Rack'].sort_values().dropna().unique())
df_racks = pd.DataFrame(df_input['IEA Rack'].sort_values().dropna().unique(), columns=['Name'])
df_racks['Make'] = ''
df_racks['Model'] = ''
df_racks['Row Label'] = df_racks['Name'].str.extract('\d(\w)-\d+')
df_racks['Position in Row'] = df_racks['Name'].str.extract('\d\w-[0]*(\d+)')
df_racks['Status'] = 'Planned'
df_racks['Capacity(kW)'] = 6
display(df_racks)
df_racks.to_csv('Output/Cabinets.csv', ',', index=False)

List of racks ['1A-02' '1A-03' '1A-04' '1A-05' '1A-08' '1A-09' '1B-01' '1B-02' '1B-03'
 '1B-04' '1B-05' '1B-06' '1B-07' '1B-08' '1B-09']


,Name,Make,Model,Row Label,Position in Row,Status,Capacity(kW)
0,1A-02,,,A,2,Planned,6
1,1A-03,,,A,3,Planned,6
2,1A-04,,,A,4,Planned,6
3,1A-05,,,A,5,Planned,6
4,1A-08,,,A,8,Planned,6
5,1A-09,,,A,9,Planned,6
6,1B-01,,,B,1,Planned,6
7,1B-02,,,B,2,Planned,6
8,1B-03,,,B,3,Planned,6
9,1B-04,,,B,4,Planned,6
